In [29]:
import time
import cv2
import HandTrackingModule as htm
import os



wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

folderPath = 'hands'
myList = os.listdir(folderPath)
# print(myList)
overlayList = []

for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlayList.append(image)
    
print(len(overlayList))
pTime = 0

detector = htm.handDetector(detectionCon=0.7)
tipIds = [4, 8, 12, 16, 20]


while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    # print(lmList)
    
    if len(lmList) !=0:
        fingers = []
        
           # thumb
        if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
            
            # 4 fingers
        for id in range(1,5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        # print(fingers)
        
        totalFingers = fingers.count(1)
        # print(totalFingers)
        
        h,w,c = overlayList[totalFingers - 1].shape 
        img[0:h, 0:w] = overlayList[totalFingers - 1] 
        
        # Display the totalFingers text only if a finger image is being displayed
        
        cv2.rectangle(img, (20, 225), (170, 425), (0,255,0), cv2.FILLED)
        cv2.putText(img, str(totalFingers), (45,375), cv2.FONT_HERSHEY_PLAIN, 10, (255,0,0), 25)

      
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (490, 25), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0 , 0), 2)
    cv2.imshow("Image", img)
    
    # Wait for 1 millisecond and check if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()  # Release the camera resource
cv2.destroyAllWindows()  # Close all OpenCV windows

6
